In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

import h5py
import os,json

In [ ]:
#change working directory
%cd ../

In [ ]:
%load_ext autoreload
%autoreload 2

from icepyx import is2class as ipd

### Choose a region for subsetting as well. Use the same region as in the core demo.

In [ ]:
region_a = ipd.Icesat2Data('ATL09',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
session=region_a.earthdata_login('liuzheng','liuzheng@apl.uw.edu')

In [ ]:
session=region_a.earthdata_login('jessica.scheick','jessica.scheick@maine.edu')

### Now, generate variable dictionary. 
Get the variable dictionary by parsing the dataset xml information from NSIDC, by calling ```_get_custom_options(session)```. 

The data variables are stored in ```region_a._cust_options['variables']```. List the options should use ```region_a.show_custom_options()``` . Currently, it calls ```_get_custom_options again, but now that all the options are stored, we should print them out directly. 

In [ ]:
opts = region_a._get_custom_options(session);
region_a.show_custom_options(session)

In [ ]:
region_a._cust_options['variables']

#### Setup the user provided variable list to subset variables
Choose ```latitude``` for ```profile_1``` only for demo purpose. 

```sc_orient``` provide info on beam strength and is stored under ```orbit_info```

In [ ]:
subset_kws = {'kw1_list':['profile_1','orbit_info'],'kw2_list':['high_rate'],'var_list':['latitude','sc_orient'],
                                             'add_default_vars':True}

### Setting params and download

In [ ]:
region_a.build_CMR_params()
region_a.build_reqconfig_params('download')

In [ ]:
region_a.subsetparams={}
region_a.build_subset_params(**subset_kws)
region_a.subsetparams

In [ ]:
region_a.order_granules(session, verbose=True)

In [ ]:
region_a.download_granules(session,'.')

### Examine downloaded subset data file 


In [ ]:
fn = '166458094/processed_ATL09_20190222003738_08490201_002_01.h5'

#### Check the downloaded dataset
Take ```latitude``` for example,

In [ ]:
varname = 'latitude'
#varname = 'sc_orient'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)
    
for tvar in varlist:
    vpath,vn = os.path.split(tvar)
    if vn==varname: print(tvar) 

#### Compare the varaible ```latitude``` in the original data and the subsetted dat

In [ ]:
varlist = region_a._cust_options['variables']
vgrp = dict()
for vn in varlist:
    vpath,vkey = os.path.split(vn)
    if vkey not in vgrp.keys():
        vgrp[vkey] = [vn]
    else:
        vgrp[vkey].append(vn)       

vgrp['latitude']